In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
# --ollama pull llama2:13b-chat
llama2_chat = ChatOllama(model="llama2:13b-chat")
llama2_code = ChatOllama(model="codellama:7b-instruct")
llm = llama2_chat


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("postgresql://localhost:5432/Seattle", sample_rows_in_table_info=0)


def get_schema(_):
    return db.get_table_info()


def run_query(query):
    return db.run(query)
print(db.get_table_info())


CREATE TABLE servicerequest (
	requestid TEXT, 
	subject TEXT, 
	requestsummary TEXT, 
	requestofficerid TEXT, 
	clientname TEXT, 
	reportedby TEXT, 
	createdon TEXT, 
	clientemail TEXT, 
	clientdepartment TEXT, 
	clientextensionattribute_4 TEXT, 
	service TEXT, 
	status TEXT, 
	incomingemail TEXT, 
	ownerteam TEXT, 
	targetfulfillment TEXT, 
	breached BOOLEAN, 
	almnumber TEXT, 
	userdefineddatetime TEXT, 
	firstescalation TEXT, 
	secondescalation TEXT, 
	thirdescalation TEXT, 
	lastmodifiedon TEXT, 
	resolvedby TEXT, 
	resolvedon TEXT, 
	closedby TEXT, 
	closedon TEXT, 
	financedeptcode TEXT, 
	financeaccount TEXT, 
	financefund TEXT, 
	financeloworg TEXT, 
	financeproject TEXT, 
	financeactivity TEXT, 
	purchasetotal TEXT, 
	vip BOOLEAN
)


In [3]:
# Prompt
from langchain_core.prompts import ChatPromptTemplate


# Update the template based on the type of SQL Database like MySQL, Microsoft SQL Server and so on
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        ("human", template),
    ]
)



In [4]:
# Chain to query
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)
print(1)
UsersQuestion = {"question": "list client Jessica Phares breached request"}
SQL = sql_response.invoke(UsersQuestion)
print(SQL)

1

SELECT * FROM servicerequest
WHERE clientname = 'Jessica Phares' AND breached = TRUE;


In [5]:
# Connect to your PostgreSQL database
# postgresql://localhost:5432/postgres
import psycopg2
conn = psycopg2.connect(
    dbname="Seattle",
    user="chenjunxu",
    # password="your_password",
    host="localhost",
    port="5432"
)

# Create a cursor object
cur = conn.cursor()

# Execute a SQL query
cur.execute(SQL)

# Fetch the results
rows = cur.fetchall()

# Do something with the results
SQL_results = [row for row in rows]
print(SQL_results)

response = llm.invoke("Very Concisely give key information: This is my question" + str(UsersQuestion.values()) 
                      + "based on my sql" + str(SQL) 
                      + "SQL result is " + str(SQL_results))
print(response.content)



# Close the cursor and connection
cur.close()
conn.close()

[('1282326', 'Mobile App Intake', 'Mobile App Intake - Bose Connect', 'x1435', 'Jessica Phares', 'Jessica Phares', '10/11/23 15:24', 'Jessica.Phares@seattle.gov', 'Community Police Commission', None, 'EUCMP - End-user Computing Service', 'Waiting for Client', 'FALSE', 'ITD_ServiceManagement', '10/25/23 15:48', True, None, None, '10/11/23 15:24', '10/18/23 15:48', '10/23/23 15:48', '2/5/24 10:00', None, None, None, None, None, None, None, None, None, None, '0.0', False)]
 Sure! Here is the key information from your SQL result:

* Client name: Jessica Phares
* Service request ID: 1282326
* Service type: Mobile App Intake
* Request status: Waiting for Client
* Breached: TRUE
* Created date: October 11, 2023 15:24
* Last updated date: October 25, 2023 15:48

I hope this helps! Let me know if you have any other questions.
